In [1]:
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
import utils
import os
import datasets
import prepare_dataset

utils.reload()

[2023-09-03 21:43:50,445] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
utils.set_gpu_ids([0, 4, 5, 6])

os.environ["CUDA_VISIBLE_DEVICES"] = 0,4,5,6


In [3]:
lr = 2e-4
batch_size = 128
num_epochs = 10

In [13]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
)

In [5]:
tokenizer = utils.get_complete_tokenizer()
model = AutoModelForCausalLM.from_pretrained(
    utils.default_model_path, low_cpu_mem_usage=True
)

head_tokens = <unk><s></s> 
tokenizer.vocab_size = 32000
len(tokenizer.vocab) = 32000
tokenizer.special_tokens_map = {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}
num_added_tokens = 1
head_tokens = <unk><s></s> 
tokenizer.vocab_size = 32000
len(tokenizer.vocab) = 32001
tokenizer.special_tokens_map = {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}
LlamaTokenizerFast(name_or_path='/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--hf-internal-testing--llama-tokenizer/snapshots/99eceeba6e8289bee767f0771166b5917e70e470', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.12433454005023165


In [7]:
training_args = TrainingArguments(
    output_dir=os.path.join(utils.models_root, "peft/test"),
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [8]:
encoded_datasets = datasets.load_from_disk(utils.encoded_datasets_path)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=utils.get_data_collator(tokenizer),
    compute_metrics=utils.compute_metrics,
)

In [16]:
trainer.train()

/data/users/zhangjunlei/anaconda3/envs/open-instruct/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 1; 79.20 GiB total capacity; 20.71 GiB already allocated; 63.31 MiB free; 20.84 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF